# Test de récupération documents avec Weaviate

### Connexion à Weaviate

In [1]:
import weaviate
from weaviate.classes.init import Auth
import os
from dotenv import load_dotenv

load_dotenv()

mistral_key = os.getenv("MISTRAL_API_KEY")
weaviate_url = os.getenv("WEAVIATE_URL")
weaviate_api_key = os.getenv("WEAVIATE_API_KEY")
headers = {
    "X-Mistral-Api-Key": mistral_key,
}

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
    headers=headers
)

collection = client.collections.use("val_de_fensch")

### Recherche vectorielle

In [2]:
vector_indexs = ["hnsw", "flat"]
vector_distances = ["cos", "dot", "l2s", "ham", "man"]

In [10]:
res = collection.query.near_text(
    query="Quels équipements culturels ont ouvert en 2008 ?",
    target_vector="vector_flat_cos",
    limit=3
)

In [11]:
for elem in res.objects:
    print(elem.properties['chunk'], "\n\n")

Bâtiments > Culture, patrimoine & tourisme > Faits marquants
- Embauche d'une chargée de projet Musiques Actuelles.
- Validation du projet architectural lors du comité de pilotage de février 2012.
- Validation du projet artistique et culturel lors du comité de pilotage de septembre 2012.
- Accord du Ministère de la Culture pour la labellisation Scène de Musiques Actuelles (SMAC).
- Démarrage des travaux en décembre 2012. 


Culture & patrimoine > Faits marquants
Ouverture du Gueulard +, nouvel équipement dédié aux musiques actuelles à Nilvange, le 14 octobre 2014. Ses missions : diffusion/création, action culturelle, ressource et accompagnement (labellisation SMAC en cours). 


Cultes > Chiffres-clés : 2014
241 212 € HT, le montant engagé en 2014 dans les édifices culturels. 




### Recherche hybride

In [18]:
from weaviate.classes.query import MetadataQuery

res = collection.query.hybrid(
    query="Quels équipements culturels ont ouvert en 2008 ?",
    target_vector="vector_flat_cos",
    alpha=1,
    limit=3,
    return_metadata=MetadataQuery(score=True, explain_score=True),
)

In [19]:
for elem in res.objects:
    print(elem.properties['chunk'], "\n\n")
    print(elem.metadata.score, elem.metadata.explain_score)

Bâtiments > Culture, patrimoine & tourisme > Faits marquants
- Embauche d'une chargée de projet Musiques Actuelles.
- Validation du projet architectural lors du comité de pilotage de février 2012.
- Validation du projet artistique et culturel lors du comité de pilotage de septembre 2012.
- Accord du Ministère de la Culture pour la labellisation Scène de Musiques Actuelles (SMAC).
- Démarrage des travaux en décembre 2012. 


1.0 
Hybrid (Result Set vector,hybridVector) Document 3b995812-6ba6-47b6-a3c6-6258ed91602d: original score 0.81715107, normalized score: 1
Culture & patrimoine > Faits marquants
Ouverture du Gueulard +, nouvel équipement dédié aux musiques actuelles à Nilvange, le 14 octobre 2014. Ses missions : diffusion/création, action culturelle, ressource et accompagnement (labellisation SMAC en cours). 


0.9271504878997803 
Hybrid (Result Set vector,hybridVector) Document 7783dea0-1e76-4435-8004-ff4df3f2a0d9: original score 0.81493795, normalized score: 0.9271505
Cultes > Chi

### Options de recherche

In [31]:
from weaviate.classes.query import Filter

res = collection.query.near_text(
    query="Quels équipements culturels ont ouvert en 2012 ?",
    target_vector="vector_flat_cos",
    limit=3,
    filters=Filter.by_property("year").equal(2012)
)

In [32]:
for elem in res.objects:
    print(elem, "\n\n")

Object(uuid=_WeaviateUUIDInt('883dee03-6c2a-45fa-82fb-b59375c879dc'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'year': 2012, 'page': 23, 'chunk': "Bâtiments > Culture, patrimoine & tourisme > Chiffres-clés : 2012\n- 113 500 € pour la préfiguration Musiques Actuelles (communication et événementiel) qui s'ajoutent à la subvention versée à l'association Pavé pour développer des missions dans le domaine des Musiques Actuelles (55 900 € en 2013).\n- 2,9 millions d'euros pour les travaux de la SMAC.", 'category': 'Bâtiments'}, references=None, vector={}, collection='Val_de_fensch') 


Object(uuid=_WeaviateUUIDInt('02ab49a2-2eeb-4ac0-a389-de9fe1c2153d'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'year': 2012, 'page':